# Counting Graph Triangles

Identifying and counting triangles in a graph is a useful way to detect communities in social networks. Triangle counts are also used to compute Local and Global Clustering Coefficients.

## Graphs, Vertices and Edges
Generally, a graph is defined as $𝐺=(𝑉,𝐸)$ where $𝑉$ is a set of vertices and $𝐸$ is a set of edges representing connections between vertices. The terms vertex and node are used interchangably, as well as the terms edge and arc. There are myriad variations of graph types.

For simplicity, the graphs used in this notebook are limited to simple graphs, i.e. graphs where there can exist at most one, undirected and un-labeled edge between any two verticies. Additionally, node labels will consist of the integers $[0 .. N-1]$ where N is the number of nodes in the graph.

## Graph Triangles

A graph triangle is a 3-node subgraph where there exists an edge between each of the three node pairs, i.e. a fully-connected 3-node subgraph. A special property of the graph adjacency matrix is used to compute the number of triangles in a simple graph.

## Graph Adjacency Matrix

A graph adjacency matrix, $A$, for an N-node graph is an NxN matrix where the rows and columns represent the graph nodes, and connections between nodes are marked with an entry of 1. For instance, if node $i$ is connected to node $j$, then a 1 will appear at $A[i,j]$.

A straightforward way to store a graph representation is as an edge list. In this file format, each line consists of two, space-separated labels representing an edge between two nodes in the graph, the first label being the edge source, the second the destination. The following function will read a simplified edge-list file (where node labels are a contiguous set of integers), and return the corresponding adjacency matrix.

The following cell defines a function that reads an edge-list file and returns an adjacency matrix.

In [1]:
import numpy as np
np.set_printoptions(precision=4, floatmode='fixed')

def read_edge_list(filename):
    #
    #   read the graph edges as node label tuples
    #
    edges = []
    with open(filename, encoding='utf-8') as f:
        for line in f.readlines():
            n1,n2 = line[:-1].split(' ')
            edges.append((int(n1),int(n2)))    
    #
    #   create a set of node labels
    #
    nodes = set()
    for edge in edges:
        nodes.add(edge[0])
        nodes.add(edge[1])
    nodes = list(nodes)
    N = len(nodes)
    #
    #   we assume that graph node labels are a set of contiguous integers
    #
    assert max(nodes) == N - 1
    #
    #   create the adjacency matrix from the list of edge tuples
    #
    A = np.zeros((N,N), dtype='int')
    A[[edge[0] for edge in edges], [edge[1] for edge in edges]] = 1

    return A


As an example, here is a simple graph. Its edge-list is stored in graphs/simple-8-node-graph-edge-list.txt.

![](images/simple-8-node-graph.png)

In [2]:
A = read_edge_list('graphs/simple-8-node-graph-edge-list.txt')
print(A)

[[0 1 0 0 0 1 0 0]
 [1 0 1 1 1 0 0 0]
 [0 1 0 1 0 0 0 0]
 [0 1 1 0 0 0 0 1]
 [0 1 0 0 0 0 1 1]
 [1 0 0 0 0 0 1 0]
 [0 0 0 0 1 1 0 1]
 [0 0 0 1 1 0 1 0]]


It is simple to test that the graph is a *simple graph*. Because graph connections are undirected, the adjacency matrix will be symmetric, and becuase there are no self-loops, the diagonal elements will be 0's.

In [3]:
assert np.all(A == A.T) and np.all(np.diagonal(A) == 0)

The adjacency matrix shows the one-hop connections between graph vertices. Squaring the matrix will show two-hop connections.

In [4]:
A_2 = np.dot(A,A)
print(A_2)

[[2 0 1 1 1 0 1 0]
 [0 4 1 1 0 1 1 2]
 [1 1 2 1 1 0 0 1]
 [1 1 1 3 2 0 1 0]
 [1 0 1 2 3 1 1 1]
 [0 1 0 0 1 2 0 1]
 [1 1 0 1 1 0 3 1]
 [0 2 1 0 1 1 1 3]]


In this example, there are two ways that node 0 can return to itself in two hops, 0-1-0 and 0-5-0. Similarly, there are four ways that node 1 can return to itself in two hops. In fact, the diagonal represents the degree (number of connections) of each node. The node degrees can also be computed by taking the marginal sums, using either axis.

Generalizing, the $P^{th}$ power of the adjacency matrix shows the number of $P$-hops exist between nodes. This property is exploited to count the number of triangles in a graph.

## Counting Triangles

The key to the algorithm is to recognize that in a graph triangle, each node is exactly three hops from itself. Taking the 3<sup>rd</sup> power of the adjacency matrix shows the number of 3-hop connections between nodes.

In [5]:
A_3 = np.dot(np.dot(A,A),A)
print(A_3)

[[0 5 1 1 1 3 1 3]
 [5 2 5 7 7 1 3 2]
 [1 5 2 4 2 1 2 2]
 [1 7 4 2 2 2 2 6]
 [1 7 2 2 2 2 5 6]
 [3 1 1 2 2 0 4 1]
 [1 3 2 2 5 4 2 5]
 [3 2 2 6 6 1 5 2]]


The diagonal elements are the number of 3-hop paths from each node back to itself, which is, by definition, the number of graph triangles that node is part of. The total number of graph triangles is computed by summing the diagonal elements. This sum is called the *trace* of the matrix in linear algebra. This sum is then divided by 6 to get the number of unique triangles because each node in a triangle has two ways to follow the triangle paths ... 3 nodes X 2 ways = 6.

In [6]:
number_of_triangles = np.trace(A_3)//6
print(f'number of triangles = {number_of_triangles}')

number of triangles = 2


Examining the graph verifies that there are indeed 2 unique triangles

![](images/simple-8-node-graph-triangles.png)

As another example, consider a more complicated, 36-node graph.

![](images/simple-36-node-graph.png)

The edge-list file for this graph is in graphs/simple-36-node-graph-edge-list.txt. Read the edge-list and compute the number of triangles from the adjacency matrix.

In [7]:
A = read_edge_list('graphs/simple-36-node-graph-edge-list.txt')
A_3 = np.dot(np.dot(A,A),A)
n_triangles = np.trace(A_3)//6
print(f'number of triangles = {n_triangles}')

number of triangles = 11


![](images/simple-36-node-graph-triangles.png)

Finally, examine the number of triangles for node 6.

In [8]:
#
#  extract the diagonal as a vector, select node 6 and divide by two (the node counts triangles for two paths)
#
np.diagonal(A_3)[6]//2

4

Node 6 is part of 4 graph triangles. 

## Local Clustering Coefficient

The local clustering coefficient indicates "how close its neighbours are to being a clique (complete graph)." (see https://en.wikipedia.org/wiki/Clustering_coefficient) The Local Clustering Coefficient for each node is computed from the formula

$$
C_i = \frac{number\,of\,closed\,triplets}{number\,of\,all\,tiplets\,(open\,and\,closed)}
$$

The *number of closed triplets* is the number of triangles that node $C_i$ is part of and the *the number of all triplets* is just the number of one-hop neigbors of $C_i$ taken 2 at a time; for undirected graphs this number is $k(k-1)/2$ where $k$ is number of one-hop neigbors. Putting this together in vectorized form:

$$
C = \frac{diagonal(A^3)/2}{(diagnonal(A^2)(diagnonal(A^2)-1)/2}\Rightarrow\frac{diagonal(A^3)}{diagnonal(A^2)(diagnonal(A^2)-1)}
$$

The following function returns the Local Clustering Coefficient for each node in the graph, given the graph adjacency matrix.

In [9]:
def local_clustering_coefficient(A):
    A_2 = np.dot(A,A)
    A_3 = np.dot(A_2, A)
    triangles = np.diagonal(A_3)
    neighbors = np.diagonal(A_2)    # can also be computed as np.sum(A, axis=0) or np.sum(A, axis=1)
    return triangles/(neighbors*(neighbors-1))

lcc = local_clustering_coefficient(A)
print(lcc)

[0.0000 0.0000 0.0000 0.0000 0.3333 0.1667 0.4000 0.3333 0.3333 0.3333
 0.3333 0.1333 1.0000 0.3333 0.0000 0.0000 0.0000 0.0000 0.2000 0.0000
 0.3333 0.2000 0.1000 0.3333 0.0000 0.0000 0.3333 0.3333 0.1667 0.2000
 0.1667 0.0000 0.0000 0.3333 0.0000 0.0000]
